In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import snscrape.modules.twitter as sntwitter
from emoji import UNICODE_EMOJI # Read documentation
import requests
import bs4

**Defining tickers**

In [81]:
ticker = "TSLA"

**Twitter scraping example**

In [82]:
max_tweets = 100
all_tweets = []
all_emojis = []

for i, tweet in enumerate(sntwitter.TwitterSearchScraper(ticker).get_items()):
    if i > max_tweets:
        break
    all_tweets.append(tweet.content)

for i in all_tweets:
    for element in i:
        if element in UNICODE_EMOJI['en']:
            all_emojis.append(element)

all_emojis[0:4]

['🤣', '🦔', '👀', '👀']

**Firstly, scraping for emoji category data from the web:**

In [83]:
url = "http://kt.ijs.si/data/Emoji_sentiment_ranking/"
response = requests.get(url)
html_parsed = bs4.BeautifulSoup(response.text, 'html.parser')


**Classifying into headers (categories) and tables (individual emojis in these categories)**

In [85]:
tables = html_parsed.find_all("tr")

**Scraping emoji symbols:**

In [86]:
emoji_list = []

for row in tables:
    string_row = str(row)
    emoji = string_row[(string_row.find("<tr><td>")+8):(string_row.find("<tr><td>")+9)]
    emoji_list.append(emoji)

**Scraping sentiment scores:**

In [87]:
sentiment_scores = []

for row in tables:
    string_row = str(row)
    if string_row[string_row.find("sentiment score: ")+17] == "-":
        score = string_row[(string_row.find("sentiment score: ")+17):(string_row.find("sentiment score: ")+23)]
    else:
        score = string_row[(string_row.find("sentiment score: ")+17):(string_row.find("sentiment score: ")+22)]
    sentiment_scores.append(score)

**Creating a dictionary:**

In [ ]:
score_dict = {}
for i in range(1,752):
    score_dict[emoji_list[i]] = sentiment_scores[i]

score_dict

**Calculating sentiment scores from actual tweets:**

In [90]:
total_score = 0

for emoji in all_emojis:
    if emoji in score_dict.keys():
        total_score = total_score + float(score_dict[emoji])

print("Current sentiment score for", ticker, "is", round(total_score, 3), ", based on last", max_tweets, "tweets")

Current sentiment score for TSLA is 8.155 , based on last 100 tweets
